In [4]:
using LinearAlgebra, CSV, Random, Tables, COSMO

In [5]:
data = CSV.File("../NC-Data.csv") |> Tables.matrix;
# Matrix sqrt: via diagonalization and sqrt eigenvalues
raw_data = sqrt(data)';
n = size(data, 1);
N = n * n;
k = 7;
# All close.
maximum(abs.(raw_data * raw_data .- data))

4.884981308350689e-15

In [6]:
nodes = CSV.File("../NC-K7-Trace-Nodes.csv") |> Tables.matrix;
bounds = CSV.File("../NC-K7-Trace-Bounds.csv") |> Tables.matrix;

In [62]:
# node_id = 50
# node_id = 300
node_id = 229
y = nodes[:, node_id]
sum(nodes[:, node_id] .== 1)

4

In [63]:
bounds[:, node_id]

3-element Array{Float64,1}:
 4.5129968872085975
 6.584373256021302
 7.0

In [64]:
selected_data = findall(nodes[:, node_id] .== 1)

4-element Array{Int64,1}:
 20
 84
 85
 86

In [65]:
s = maximum(svd(raw_data[:, selected_data]).S)
u = svd(raw_data[:, selected_data]).U[:, 1];
v = svd(raw_data[:, selected_data]).V[:, 1];
D = Matrix(1.0I, 101, 101)[:, selected_data];
s

1.9342690514902823

In [66]:
rank_one_term = (raw_data * u) .^ 2
rank_one_term[nodes[:, node_id] .== 0] .= 0
rank_one_term

101-element Array{Float64,1}:
 0.0011302640621757621
 0.02770066314539759
 0.10151772407545055
 0.0
 0.25610915747693164
 0.00013428537660249162
 0.09003700443881506
 0.05574370907683892
 0.05354760607565086
 0.053561867714760604
 2.3440104777510103e-5
 0.15284319464378268
 0.0
 ⋮
 0.0027394360743202323
 7.469439224815266e-5
 0.18866496469341965
 0.1484103392065695
 0.016873719861435224
 0.0073537692505327885
 0.01652816329702802
 0.002684390237879137
 0.028361399876088253
 0.1262101742004988
 0.00255421792241594
 0.0

In [67]:
# Upper bound on the contribution from adding one variable into ||E||^2.
# The upper bound comes from ||E||_F^2
# upper bounded by ||(I-uu')MD||_F^2. Also, we know that normCommunities
# has column norm of 1 for every variable.
# We should add another upper bound based on taking k columns from this
# row into ||E||_F.
residual_term = 1 .- rank_one_term

101-element Array{Float64,1}:
 0.9988697359378242
 0.9722993368546025
 0.8984822759245494
 1.0
 0.7438908425230684
 0.9998657146233975
 0.909962995561185
 0.9442562909231611
 0.9464523939243491
 0.9464381322852394
 0.9999765598952225
 0.8471568053562173
 1.0
 ⋮
 0.9972605639256797
 0.9999253056077518
 0.8113350353065804
 0.8515896607934305
 0.9831262801385647
 0.9926462307494672
 0.983471836702972
 0.9973156097621209
 0.9716386001239118
 0.8737898257995012
 0.997445782077584
 1.0

In [68]:
function frobenius_rows_k(M, y)
    M = M .^ 2
    row_quantities = zeros(n)
    stillneed = k - sum(y .== 1)
    for row_index in 1:n
        if y[row_index] == 0
            continue
        end
        row = copy(M[row_index, :])
        row_sum = sum(row[y .== 1])
        row[y .== 1] .= 0
        if y[row_index] == -1
            row_sum += row[row_index]
            row[row_index] = 0
            row_stillneed = stillneed -1
        else
            row_stillneed = stillneed
        end
        row_sum += sum(sort(row, rev=true)[1:row_stillneed])
        row_quantities[row_index] = row_sum
    end
    row_quantities
end

frobenius_rows_k (generic function with 1 method)

In [69]:
frobenius_rows_k(data - raw_data * u * u' * raw_data, y)

101-element Array{Float64,1}:
 2.8791477009350745
 2.368268229536578
 1.8399238146309513
 0.0
 1.0799128608138524
 2.6450376893478134
 2.0739675409026956
 2.2559632852536087
 2.3531832913991217
 2.13404028124117
 2.871778198643323
 0.9300107211590581
 0.0
 ⋮
 2.342129587899845
 1.9628366529875065
 2.02614301670143
 1.5229829966217263
 2.118548378912177
 2.1845508923663695
 2.0119418404296447
 1.9848715159967312
 1.621183029679441
 1.2325725671930565
 1.4463939058549293
 0.0

In [70]:
sum((y .== -1) .& (frobenius_rows_k(data - raw_data * u * u' * raw_data, y) .< residual_term))

9

In [85]:
stillneed = k - sum(y .== 1)
lb_proj_nodes = rank_one_term .>= sort(rank_one_term .* Array{Float64}(y .== -1))[end-stillneed+1]
lb_proj_nodes = lb_proj_nodes .& (y .== -1)
lb_proj_nodes = lb_proj_nodes .| (y .== 1)
lb_proj = sum(rank_one_term[lb_proj_nodes])
lb_diag = maximum(svd(raw_data[:, lb_proj_nodes]).S) ^ 2

6.382260199468312

In [72]:
bounds[:, node_id]

3-element Array{Float64,1}:
 4.5129968872085975
 6.584373256021302
 7.0

In [73]:
lambda_1 = bounds[2, node_id]
lambda_2 = bounds[3, node_id] - lb_diag
lambda_2_lb = bounds[3, node_id] - lambda_1
lambda_2

0.6177398005316883

In [74]:
model = COSMO.Model()

# Number of auxiliary helper variables
# v[n+1] -> sqrt(x(1-x))
aux = n+1

ones_c = COSMO.Constraint(
    [Vector{Float64}(y .== 1)' 0],
    [-Float64(sum(y .== 1))],
    COSMO.ZeroSet)

zeros_c = COSMO.Constraint(
    [Vector{Float64}(y .== 0)' 0],
    [0.],
    COSMO.ZeroSet)

nonnegative_c = COSMO.Constraint(
    Matrix{Float64}(I, aux, aux),
    zeros(aux),
    COSMO.Nonnegatives)

box_c = COSMO.Constraint(
    [I zeros(n, 1)],
    zeros(n),
    COSMO.Box(zeros(n), ones(n)))

trace_c = COSMO.Constraint(
    [ones(1, n) 0.],
    [-Float64(k)],
    COSMO.ZeroSet)

x_e = residual_term' / lambda_1^2
# sqrt(x(1-x)) >= x_3
power_cone = [
    x_e 0;
    -x_e 0;
    zeros(1, n) 1
]
power_cone = COSMO.Constraint(
    power_cone,
    [0.; 1; 0],
    COSMO.PowerCone(0.5))

if (false)
    assemble!(
        model,
        zeros(aux, aux),
        [-(x_e .* lambda_2) -sqrt(lambda_1 * lambda_2)],
        [ones_c, zeros_c, nonnegative_c, box_c, trace_c, power_cone])
else
    assemble!(
        model,
        zeros(aux, aux),
        [-(x_e .* lambda_1 + rank_one_term') -sqrt(lambda_1 * lambda_2)],
        [ones_c, zeros_c, nonnegative_c, box_c, trace_c, power_cone])
end

In [75]:
result = COSMO.optimize!(model)

>>> COSMO - Results
Status: 

Solved
Iterations: 127 (incl. 27 safeguarding iterations)
Optimal Objective: -6.646
Runtime: 134.88ms
Setup Time: 0.25ms

Avg Iter Time: 1.06ms

In [53]:
result.x

102-element Array{Float64,1}:
  5.862393108023638e-18
  7.475917501669558e-16
  8.298090657497154e-16
  1.9129431112693426e-16
 -1.0957623803792581e-16
 -6.181672997169063e-16
 -1.6639866634306561e-16
  6.062010593185355e-17
 -1.2729208001449343e-15
 -1.5486059870229226e-17
 -3.8514703043622913e-16
  2.7578503070455356e-16
  1.9129431112693426e-16
  ⋮
 -2.4767984999566298e-17
  1.0608597181916221e-16
  8.602376074086254e-17
 -5.259807861667811e-16
 -7.515349368682203e-16
 -6.308449706801816e-16
 -4.51244848725479e-16
 -1.7545301833236387e-15
 -3.908748658355219e-16
 -4.8305561488238045e-16
  1.9129431112693426e-16
  0.13135371544821212

In [81]:
maximum(svd(raw_data .* result.x[1:n]).S .^ 2)

6.382259002781803

In [82]:
maximum(svd((raw_data * (I - u * u')) .* result.x[1:n]).S .^ 2)

0.35429099638255307

In [83]:
sqrt(residual_term' * result.x[1:n])

0.8496980351579518